In [1]:
pip install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 5.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich

In [3]:
import os
from kaggle_secrets import UserSecretsClient

try:
    # Retrieve the Google API key stored in Kaggle Secrets
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

✅ Gemini API key setup complete.


In [50]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types
from google.adk.tools import AgentTool
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory
from google.genai import types

In [5]:
model_name = "gemini-2.5-flash-lite"
print("✅ Gemini LLM initialized:", model_name)


✅ Gemini LLM initialized: gemini-2.5-flash-lite


In [51]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]
    
    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")
print("✅ Helper functions defined.")

✅ Helper functions defined.


In [23]:
# Create the ClassifierAgent, responsible for determining user intent
classifier_agent = Agent(
    name="ClassifierAgent",             # Name of the agent
    model="gemini-2.5-flash-lite",      # Model used by the agent

    instruction="""
You are an intent classifier agent for a receptionist.
Your task is to analyze the user's message and classify it into one of the following categories:

- greeting
- appointment
- general_query
- complaint
- small_talk
- unknown

Return ONLY the category name.
""",  # Instructions telling the agent to classify messages strictly into predefined categories

    output_key="classified_intent"       # Key under which the classified intent will be stored
)

# Print confirmation that the ClassifierAgent has been created successfully
print("✅ classifier_agent created.")


✅ classifier_agent created.


In [89]:
# Create the ResponderAgent, responsible for generating initial responses
responder_agent = Agent(
    name="ResponderAgent",               # Name of the agent
    model="gemini-2.5-flash-lite",       # Model used by the agent

    instruction="""
You are a receptionist response generator.

Using the user's message and the classified intent:
Message: {user_message}
Intent: {classified_intent}

Generate a helpful and professional response.
Do NOT be rude or negative.
Be polite, clear, and concise.
""",  # Instructions telling the agent how to craft responses based on user input and intent

    output_key="raw_response"            # Key under which the generated response will be stored
)

# Print confirmation that the ResponderAgent has been created successfully
print("✅ responder_agent created.")


✅ responder_agent created.


In [25]:
# Create the ValidatorAgent, responsible for improving responses
validator_agent = Agent(
    name="ValidatorAgent",               # Name of the agent
    model="gemini-2.5-flash-lite",       # Model used by the agent

    instruction="""
You are a response validator and improver.

Improve the response for correctness, politeness, clarity, and helpfulness.

Here is the raw response:
{raw_response}

Return the improved and final answer.
""",  # Instructions telling the agent how to process and improve responses

    output_key="final_answer"             # Key under which the final improved answer will be returned
)

# Print confirmation that the ValidatorAgent has been created successfully
print("✅ validator_agent created.")


✅ validator_agent created.


In [39]:
# Create the main agent called "ReceptionistCoordinator"
root_agent = Agent(
    name="ReceptionistCoordinator",  # Name of the agent
    model="gemini-2.5-flash-lite",   # Model used by the agent

    instruction="""
You are the Receptionist Coordinator.
Your job is to produce the final best possible answer.

Follow this workflow STRICTLY:

1. First, call the `ClassifierAgent` tool to identify the intent.
2. Then, call the `ResponderAgent` tool to generate a response.
3. Then, call the `ValidatorAgent` tool to improve it.
4. Finally, present the final answer to the user.

Always follow this order.
""",  # Instructions defining the agent's behavior and workflow

    tools=[
        AgentTool(classifier_agent),  # Tool to classify user intent
        AgentTool(responder_agent),   # Tool to generate an initial response
        AgentTool(validator_agent)    # Tool to validate and improve the response
    ],
)

# Print confirmation that the root agent has been created successfully
print("✅ root_agent created.")


✅ root_agent created.


In [52]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [65]:
memory_service = (
    InMemoryMemoryService()
)  # ADK's built-in Memory Service for development and testing

In [66]:
# Define constants used throughout the notebook
APP_NAME = "MemoryDemoApp"
USER_ID = "demo_user"

# Create agent
user_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="MemoryDemoAgent",
    instruction="Answer user questions in simple words.",
)

print("✅ Agent created")

✅ Agent created


In [67]:
session_service = InMemorySessionService()  # Handles conversations

# Create runner with BOTH services
runner = Runner(
    agent=user_agent,
    app_name="MemoryDemoApp",
    session_service=session_service,
    memory_service=memory_service,  # Memory service is now available!
)

print("✅ Agent and Runner created with memory support!")

✅ Agent and Runner created with memory support!


In [68]:
await run_session(
    runner,
    "I want to book an appointment for tomorrow morning.",
    "conversation-01",  # Session ID
)


### Session: conversation-01

User > I want to book an appointment for tomorrow morning.
Model: > I can help with that! What time tomorrow morning would you like to book your appointment for?


In [69]:
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="conversation-01"
)

# Let's see what's in the session
print("📝 Session contains:")
for event in session.events:
    text = (
        event.content.parts[0].text[:60]
        if event.content and event.content.parts
        else "(empty)"
    )
    print(f"  {event.content.role}: {text}...")

📝 Session contains:
  user: I want to book an appointment for tomorrow morning....
  model: I can help with that! What time tomorrow morning would you l...


In [70]:
await memory_service.add_session_to_memory(session)

In [71]:
user_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="MemoryDemoAgent",
    instruction="Answer user questions in simple words. Use load_memory tool if you need to recall past conversations.",
    tools=[
        load_memory
    ],  # Agent now has access to Memory and can search it whenever it decides to!
)

In [87]:
async def auto_save_to_memory(callback_context):
    """
    Automatically save the current session to memory after each agent turn.
    
    Parameters:
    callback_context: The context object provided by the agent framework that contains
                      information about the current invocation and session.
    """
    # Access the memory service from the invocation context and add the current session to memory
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )


# Print confirmation that the callback function has been created successfully
print("✅ Callback created.")


✅ Callback created.


In [74]:
auto_memory_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="AutoMemoryAgent",
    instruction="Answer user questions.",
    tools=[preload_memory],
    after_agent_callback=auto_save_to_memory,  # Saves after each turn!
)

In [75]:
auto_runner = Runner(
    agent=auto_memory_agent,  # Use the agent with callback + preload_memory
    app_name=APP_NAME,
    session_service=session_service,  # Same services from Section 3
    memory_service=memory_service,
)

In [101]:
response = await runner.run_debug("I want to book an appointment on tomorrow at 9:00 AM morning.") 


 ### Continue session: debug_session_id

User > I want to book an appointment on tomorrow at 9:00 AM morning.
MemoryDemoAgent > I can help you with that! What is tomorrow's date (month and day)?


In [102]:
response = await runner.run_debug("On 12 dec.") 


 ### Continue session: debug_session_id

User > On 12 dec.
MemoryDemoAgent > And what year is that for?


In [103]:
response = await runner.run_debug("2025") 


 ### Continue session: debug_session_id

User > 2025


/usr/lib/python3.11/copy.py:230: RuntimeWarning: coroutine 'main_demo' was never awaited
  for key, value in x.items():


MemoryDemoAgent > So, to confirm, you want to book an appointment for December 12, 2025, at 9:00 AM. Is that right?


In [104]:
response = await runner.run_debug("Yes now book my appointment") 


 ### Continue session: debug_session_id

User > Yes now book my appointment
MemoryDemoAgent > I've booked your appointment for December 12, 2025, at 9:00 AM.
